<a href="https://colab.research.google.com/github/ArshT/Pytorch_Practice/blob/master/Pytorch_CIFAR_10_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim

In [3]:
transform_train = transforms.Compose([transforms.Resize(96),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomRotation(degrees=(-30, 30)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

transform_test = transforms.Compose([transforms.Resize(96),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(trainset,batch_size = 32,shuffle=True,num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset,batch_size = 4,shuffle=False,num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')

device = torch.device("cuda:0" if USE_GPU else "cpu")

using device: cuda


In [8]:
backbone_model = models.alexnet(pretrained=True)
for param in backbone_model.parameters():
      param.requires_grad = True

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.features = nn.Sequential(*list(backbone_model.features.children()))
    self.fc1 = nn.Linear(256*2*2,10)

  def forward(self,x):

    x = self.features(x)
    x = x.view(x.shape[0],256*2*2)
    x = self.fc1(x)

    return x       

net = Net()
net = net.to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9)

In [10]:
def model_eval(dataloader,model):
    total = 0
    correct = 0
    for data in dataloader:
      images, l = data
      
      images = images.to(device)
      l = l.to(device)
      
      out = model(images)
      max_val, preds = torch.max(out,dim=1)
      
      total += l.shape[0]                   
      correct += (preds == l).sum().item()  
      accuracy = (100 * correct)/total
    
    
    return accuracy

In [11]:
for epoch in range(15):
  print("Epoch:",epoch+1)
  running_loss = 0.0
  for i,data in enumerate(trainloader,0):
    inputs,labels = data
    
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    
    output = net(inputs)
    output = output.to(device)
    loss = criterion(output, labels)
    
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    
    
    if i % 300 == 299:    
      print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 300))
      running_loss = 0.0
  
  print("EPOCH OVER")
  train_acc = model_eval(trainloader,net)
  test_acc = model_eval(testloader,net)
  print("############################")
  print("Training Accuracy:",train_acc,"Testing Accuracy",test_acc)
  print("############################")
  print("")

Epoch: 1
[1,   300] loss: 1.257
[1,   600] loss: 0.907
[1,   900] loss: 0.859
[1,  1200] loss: 0.780
[1,  1500] loss: 0.751
EPOCH OVER
############################
Training Accuracy: 75.474 Testing Accuracy 77.45
############################

Epoch: 2
[2,   300] loss: 0.689
[2,   600] loss: 0.683
[2,   900] loss: 0.682
[2,  1200] loss: 0.631
[2,  1500] loss: 0.633
EPOCH OVER
############################
Training Accuracy: 79.294 Testing Accuracy 80.03
############################

Epoch: 3
[3,   300] loss: 0.578
[3,   600] loss: 0.588
[3,   900] loss: 0.570
[3,  1200] loss: 0.568
[3,  1500] loss: 0.564
EPOCH OVER
############################
Training Accuracy: 80.772 Testing Accuracy 80.21
############################

Epoch: 4
[4,   300] loss: 0.511
[4,   600] loss: 0.512
[4,   900] loss: 0.521
[4,  1200] loss: 0.524
[4,  1500] loss: 0.522
EPOCH OVER
############################
Training Accuracy: 83.562 Testing Accuracy 83.2
############################

Epoch: 5
[5,   300] loss: 0.4

In [18]:
vgg16_backbone = models.vgg16(pretrained=True)

for param in vgg16_backbone.parameters():
      param.requires_grad = True

class VGG16(nn.Module):
  def __init__(self):
    super(VGG16, self).__init__()
    self.features = nn.Sequential(*list(vgg16_backbone.features.children()))

    self.drop1 = nn.Dropout(0.3)
    self.drop2 = nn.Dropout(0.2)

    self.fc1 = nn.Linear(512*3*3,1024)
    self.fc2 = nn.Linear(1024,512)
    self.fc3 = nn.Linear(512,256)
    self.fc4 = nn.Linear(256,128)
    self.fc5 = nn.Linear(128,64)
    self.fc6 = nn.Linear(64,10)

  def forward(self,x):

    x = self.features(x)

    x = x.view(x.shape[0],512*3*3)
    
    x = F.relu(self.fc1(x))
    x = self.drop1(x)
    x = F.relu(self.fc2(x))
    x = self.drop1(x)
    x = F.relu(self.fc3(x))
    x = self.drop1(x)
    x = F.relu(self.fc4(x))
    x = self.drop2(x)
    x = F.relu(self.fc5(x))
    x = self.drop2(x)
    x = self.fc6(x)

    return x       

vgg_model = VGG16()
vgg_model = vgg_model.to(device)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg_model.parameters(), lr=0.003,momentum=0.9)

In [20]:
for epoch in range(10):
  print("Epoch:",epoch+1)
  running_loss = 0.0
  for i,data in enumerate(trainloader,0):
    inputs,labels = data
    
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    
    output = vgg_model(inputs)
    output = output.to(device)
    loss = criterion(output, labels)
    
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    
    
    if i % 300 == 299:    
      print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 300))
      running_loss = 0.0
  
  print("EPOCH OVER")
  train_acc = model_eval(trainloader,vgg_model)
  test_acc = model_eval(testloader,vgg_model)
  print("############################")
  print("Training Accuracy:",train_acc,"Testing Accuracy",test_acc)
  print("############################")
  print("")

Epoch: 1
[1,   300] loss: 2.243
[1,   600] loss: 1.385
[1,   900] loss: 0.920
[1,  1200] loss: 0.738
[1,  1500] loss: 0.693
EPOCH OVER
############################
Training Accuracy: 83.856 Testing Accuracy 85.73
############################

Epoch: 2
[2,   300] loss: 0.583
[2,   600] loss: 0.566
[2,   900] loss: 0.522
[2,  1200] loss: 0.491
[2,  1500] loss: 0.460
EPOCH OVER
############################
Training Accuracy: 85.984 Testing Accuracy 87.09
############################

Epoch: 3
[3,   300] loss: 0.434
[3,   600] loss: 0.396
[3,   900] loss: 0.410
[3,  1200] loss: 0.388
[3,  1500] loss: 0.368
EPOCH OVER
############################
Training Accuracy: 91.03 Testing Accuracy 90.79
############################

Epoch: 4
[4,   300] loss: 0.334
[4,   600] loss: 0.339
[4,   900] loss: 0.335
[4,  1200] loss: 0.331
[4,  1500] loss: 0.340
EPOCH OVER
############################
Training Accuracy: 91.58 Testing Accuracy 91.32
############################

Epoch: 5
[5,   300] loss: 0.28

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [23]:
PATH = '/content/gdrive/My Drive/Pytorch_Practice/CIFAR-10_Transfer_Learning_Model-1.pt'
torch.save(vgg_model,PATH)